## dividir imagen de alta resolucion de tablero

In [3]:
from PIL import Image
import numpy as np

def split_image(image_path, tile_size):
    image = Image.open(image_path)
    width, height = image.size
    tiles = []

    for top in range(0, height, tile_size):
        for left in range(0, width, tile_size):
            box = (left, top, min(left + tile_size, width), min(top + tile_size, height))
            tiles.append((box, image.crop(box)))
    
    return tiles


## aplicar modelo a las sub imagenes obtenidas

In [13]:
from ultralytics import YOLO

# Cargar el modelo YOLOv8
model = YOLO('last.pt')

from tqdm import tqdm

def apply_model_to_tiles(model, tiles):
    results = []
    for box, tile in tqdm(tiles, desc="Processing tiles"):  # Uso correcto de tqdm.tqdm
        tile_array = np.array(tile)  # Convertir la subimagen a formato numpy
        result = model(tile_array)  # Aplicar el modelo
        results.append((box, result))
    return results


## re ajustar coordenadas de las detecciones

In [15]:
def adjust_detections_to_global_space(tiles_with_detections):
    global_detections = []
    for (box, result) in tiles_with_detections:
        left, top, _, _ = box  # Coordenadas de la subimagen
        for detection in result[0].boxes:  # Acceder a las detecciones
            # Obtener las coordenadas de la detección
            x1, y1, x2, y2 = detection.xyxy[0].tolist()  # Convertir a lista
            label = detection.cls.item()  # Clase detectada como entero
            conf = detection.conf.item()  # Confianza de la detección
            # Ajustar las coordenadas al espacio global
            global_detections.append({
                'x1': x1 + left, 'y1': y1 + top,
                'x2': x2 + left, 'y2': y2 + top,
                'label': int(label),  # Clase detectada
                'confidence': conf   # Confianza
            })
    return global_detections


## recombinar resultados

In [39]:
import cv2
import tqdm

def visualize_detections(image_path, global_detections, class_names, output_path='output_with_detections.jpg'):
    # Cargar la imagen original
    image = cv2.imread(image_path)

    # Dibujar cada detección en la imagen
    for detection in global_detections:
        # Acceder a las coordenadas y la clase desde el diccionario
        x1, y1, x2, y2 = int(detection['x1']), int(detection['y1']), int(detection['x2']), int(detection['y2'])
        class_id = int(detection['label'])  # ID de la clase
        confidence = detection['confidence']

        # Mostrar solo las detecciones con confianza mayor a 0.65
        if confidence >= 0.55:
            # Obtener el nombre de la clase
            class_name = class_names[class_id] if class_id < len(class_names) else f"Class {class_id}"

            # Dibujar un rectángulo alrededor del objeto detectado
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Ajustar el texto y el color
            text = f"{class_name} ({confidence:.2f})"
            font_scale = 0.7  # Aumentar el tamaño de la fuente
            font_thickness = 2  # Grosor de la fuente
            text_color = (0, 0, 139)  # Rojo oscuro en formato BGR

            # Dibujar el texto
            text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
            text_w, text_h = text_size
            label_y = max(y1 - 10, text_h + 10)  # Asegurar que el texto no salga del borde superior

            cv2.putText(image, text, (x1, label_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness)

    # Guardar la imagen con las detecciones visualizadas
    cv2.imwrite(output_path, image)
    print(f"Detections saved to {output_path}")

In [14]:
if __name__ == "__main__":
    # Ruta de la imagen grande
    image_path = "prueba.jpg"

    # Cargar el modelo YOLOv8 entrenado
    model = YOLO('last.pt')

    # Dividir la imagen en subimágenes
    tile_size = 640  # Tamaño de cada subimagen
    tiles = split_image(image_path, tile_size)



Processing tiles:   0%|          | 0/416 [00:00<?, ?it/s]


0: 640x640 (no detections), 41.6ms
Speed: 1.7ms preprocess, 41.6ms inference, 5.5ms postprocess per image at shape (1, 3, 640, 640)


Processing tiles:   0%|          | 1/416 [00:01<13:41,  1.98s/it]


0: 640x640 (no detections), 41.0ms
Speed: 1.0ms preprocess, 41.0ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 36.3ms
Speed: 0.9ms preprocess, 36.3ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 36.3ms
Speed: 0.9ms preprocess, 36.3ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)


Processing tiles:   1%|          | 4/416 [00:02<02:47,  2.45it/s]

...


0: 352x640 (no detections), 21.2ms
Speed: 0.6ms preprocess, 21.2ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 21.2ms
Speed: 0.6ms preprocess, 21.2ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 21.2ms
Speed: 0.5ms preprocess, 21.2ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 21.8ms
Speed: 0.6ms preprocess, 21.8ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 21.5ms
Speed: 0.6ms preprocess, 21.5ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)


Processing tiles:  99%|█████████▉| 411/416 [00:17<00:00, 38.55it/s]


0: 352x640 (no detections), 21.3ms
Speed: 0.6ms preprocess, 21.3ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 21.1ms
Speed: 0.6ms preprocess, 21.1ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 transformador-corriente, 21.2ms
Speed: 0.6ms preprocess, 21.2ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 3 proteccion-termomagneticas, 21.3ms
Speed: 0.6ms preprocess, 21.3ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 640)

0: 384x640 (no detections), 34.0ms
Speed: 0.9ms preprocess, 34.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)


Processing tiles: 100%|██████████| 416/416 [00:18<00:00, 23.03it/s]


AttributeError: 'Results' object has no attribute 'x'. See valid attributes below.

    A class for storing and manipulating inference results.

    This class encapsulates the functionality for handling detection, segmentation, pose estimation,
    and classification results from YOLO models.

    Attributes:
        orig_img (numpy.ndarray): Original image as a numpy array.
        orig_shape (Tuple[int, int]): Original image shape in (height, width) format.
        boxes (Boxes | None): Object containing detection bounding boxes.
        masks (Masks | None): Object containing detection masks.
        probs (Probs | None): Object containing class probabilities for classification tasks.
        keypoints (Keypoints | None): Object containing detected keypoints for each object.
        obb (OBB | None): Object containing oriented bounding boxes.
        speed (Dict[str, float | None]): Dictionary of preprocess, inference, and postprocess speeds.
        names (Dict[int, str]): Diction

In [16]:
    # Aplicar el modelo a cada subimagen
    tiles_with_detections = apply_model_to_tiles(model, tiles)



Processing tiles:   0%|          | 0/416 [00:00<?, ?it/s]


0: 640x640 (no detections), 42.0ms
Speed: 9.4ms preprocess, 42.0ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 41.6ms
Speed: 1.0ms preprocess, 41.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)


Processing tiles:   0%|          | 2/416 [00:00<00:20, 19.98it/s]


0: 640x640 (no detections), 38.7ms
Speed: 0.9ms preprocess, 38.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 37.7ms
Speed: 1.0ms preprocess, 37.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 37.7ms
Speed: 1.0ms preprocess, 37.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)


Processing tiles:   1%|          | 5/416 [00:00<00:18, 22.81it/s]

...


0: 352x640 (no detections), 21.2ms
Speed: 0.8ms preprocess, 21.2ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 21.3ms
Speed: 0.7ms preprocess, 21.3ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 21.8ms
Speed: 0.7ms preprocess, 21.8ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 21.6ms
Speed: 0.9ms preprocess, 21.6ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 (no detections), 21.3ms
Speed: 0.7ms preprocess, 21.3ms inference, 0.3ms postprocess per image at shape (1, 3, 352, 640)


Processing tiles:  99%|█████████▉| 413/416 [00:16<00:00, 38.81it/s]


0: 352x640 1 transformador-corriente, 21.3ms
Speed: 0.8ms preprocess, 21.3ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 3 proteccion-termomagneticas, 21.2ms
Speed: 0.8ms preprocess, 21.2ms inference, 0.6ms postprocess per image at shape (1, 3, 352, 640)

0: 384x640 (no detections), 22.9ms
Speed: 1.0ms preprocess, 22.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)


Processing tiles: 100%|██████████| 416/416 [00:16<00:00, 25.70it/s]


AttributeError: 'dict' object has no attribute 'x1'

In [17]:
    # Ajustar las detecciones al espacio global
    global_detections = adjust_detections_to_global_space(tiles_with_detections)


In [47]:
# Visualizar las detecciones en la imagen completa
class_names = [
    'actuador-knx', 'analizador-red', 'bloqueo', 'borne', 'borne-cortocircuitable',
    'contactor', 'enclavamiento', 'equipo-de-medida', 'fuente-knx', 'fusible',
    'interruptor-puerta', 'lampara', 'luz-piloto', 'mando-motorizado', 'proteccion-diferencial',
    'proteccion-sobretension', 'proteccion-termomagnetica', 'proteccion-termomagnetica-extraible',
    'puerto-comunicacion', 'seccionador', 'seccionador-extraible', 'selector',
    'transformador-aislacion', 'transformador-corriente', 'ventilador'
]
#abreviadas
class_names = [
        'act-knx', 'ana-red', 'bloqueo', 'borne', 'borne-cc',
        'contactor', 'enclav', 'eq-medida', 'fuente-knx', 'fusible',
        'int-puerta', 'lampara', 'luz-piloto', 'mando-motor', 'prot-diff',
        'prot-sobre', 'prot-term', 'prot-term-ext', 'pto-com', 'seccionador',
        'secc-ext', 'selector', 'trafo-aisl', 'trafo-corr', 'ventilador'
    ]


visualize_detections(image_path, global_detections, class_names)


Detections saved to output_with_detections.jpg


## consolidado de detecciones

In [43]:
import pandas as pd
def consolidate_detections(global_detections, class_names):

    # Inicializar un contador para cada clase
    class_counts = {class_name: 0 for class_name in class_names}

    # Contar las detecciones por clase
    for detection in global_detections:
        class_id = int(detection['label'])  # ID de la clase
        if class_id < len(class_names):  # Verificar que el ID esté en el rango válido
            class_name = class_names[class_id]
            class_counts[class_name] += 1

    # Crear un DataFrame a partir del diccionario y ordenarlo
    df = pd.DataFrame(list(class_counts.items()), columns=["Class", "Count"])
    df = df.sort_values(by="Count", ascending=False).reset_index(drop=True)

    return df

In [48]:
consolidated_df = consolidate_detections(global_detections, class_names)
# Mostrar el DataFrame

print(consolidated_df)

                                  Class  Count
0                proteccion-diferencial    212
1             proteccion-termomagnetica    165
2                                 borne    137
3                               fusible     19
4                   puerto-comunicacion     16
5               transformador-corriente     16
6                      equipo-de-medida     15
7                            luz-piloto     14
8                borne-cortocircuitable      9
9                            ventilador      9
10                              lampara      8
11                              bloqueo      6
12                           fuente-knx      5
13                   interruptor-puerta      4
14              transformador-aislacion      3
15                         actuador-knx      3
16                       analizador-red      1
17                            contactor      1
18                        enclavamiento      1
19                          seccionador      1
20           